# Lab 10: Dataset Frankenstein — budujemy dane do ML

W tym notebooku zbudujemy dataset do predykcji defektów.
Każdy wiersz to plik `.py`, kolumny to metryki kodu, a ostatnia kolumna mówi czy plik jest "buggy" czy "clean".

## 1. Importy i konfiguracja

In [ ]:
import subprocess
import os
from pathlib import Path
from datetime import datetime, date
from collections import Counter, defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from radon.complexity import cc_visit

# Konfiguracja wykresów
plt.rcParams["figure.figsize"] = (12, 6)
sns.set_style("whitegrid")

# Ścieżka do repozytorium do analizy
REPO_PATH = "/tmp/requests"  # <-- zmień na swoją ścieżkę

# Słowa kluczowe do wykrywania commitów naprawiających bugi
BUG_KEYWORDS = ["fix", "bug", "error", "fault", "defect", "patch", "repair", "crash", "issue"]

## 2. Zbieranie listy plików .py

Znajdźcie wszystkie pliki `.py` w projekcie. Pomińcie katalogi z testami, `setup.py`, `conftest.py` i inne pliki konfiguracyjne.

In [ ]:
def find_python_files(repo_path: str) -> list[str]:
    """Znajdź wszystkie pliki .py w repozytorium (bez testów)."""
    exclude = {"test", "tests", "testing", "__pycache__", ".tox", ".eggs"}
    exclude_files = {"setup.py", "conftest.py", "noxfile.py", "tasks.py"}
    
    py_files = []
    for path in Path(repo_path).rglob("*.py"):
        # Pomiń katalogi z testami
        parts = set(path.relative_to(repo_path).parts)
        if parts & exclude:
            continue
        if path.name in exclude_files:
            continue
        py_files.append(str(path.relative_to(repo_path)))
    
    return sorted(py_files)

py_files = find_python_files(REPO_PATH)
print(f"Znaleziono {len(py_files)} plików .py")
for f in py_files:
    print(f"  {f}")

## 3. Metryki produktowe: LOC i złożoność cyklomatyczna

Dla każdego pliku obliczamy:
- **LOC** — liczba niepustych linii
- **avg_cc** — średnia złożoność cyklomatyczna (z radon)
- **max_cc** — maksymalna CC w pliku
- **num_functions** — liczba funkcji/metod

In [ ]:
def compute_loc(filepath: str) -> int:
    """Policz niepuste linie kodu (bez komentarzy)."""
    # TODO: Zaimplementuj liczenie LOC
    # Wskazówka: otwórz plik, pomiń puste linie i linie z samym komentarzem (#)
    pass


def compute_complexity(filepath: str) -> dict:
    """Oblicz metryki złożoności cyklomatycznej z radon."""
    # TODO: Zaimplementuj obliczanie CC
    # Wskazówka: użyj radon.complexity.cc_visit(source_code)
    # Zwróć dict z kluczami: avg_cc, max_cc, num_functions
    pass


# Zbierz metryki produktowe
product_metrics = []

for pyfile in py_files:
    filepath = os.path.join(REPO_PATH, pyfile)
    try:
        loc = compute_loc(filepath)
        cc = compute_complexity(filepath)
        product_metrics.append({
            "filename": pyfile,
            "loc": loc,
            "avg_cc": cc["avg_cc"],
            "max_cc": cc["max_cc"],
            "num_functions": cc["num_functions"],
        })
    except Exception as e:
        print(f"Błąd przy {pyfile}: {e}")

df_product = pd.DataFrame(product_metrics)
print(f"Zebrano metryki produktowe dla {len(df_product)} plików")
df_product.head(10)

## 4. Metryki procesowe: churn, autorzy, wiek

Z `git log --numstat` wyciągamy:
- **churn** — suma dodanych i usuniętych linii we wszystkich commitach
- **num_commits** — ile razy plik był zmieniany
- **num_authors** — ilu różnych autorów dotknęło pliku
- **age_days** — ile dni temu plik pojawił się w repozytorium

In [ ]:
def parse_git_log(repo_path: str) -> list[dict]:
    """Parsuj git log --numstat i zwróć listę commitów."""
    result = subprocess.run(
        ["git", "log", "--numstat",
         "--format=%H|%an|%ad|%s", "--date=short"],
        cwd=repo_path,
        capture_output=True, text=True, check=True,
        encoding="utf-8", errors="replace",
    )

    commits = []
    current = None

    for line in result.stdout.split("\n"):
        line = line.strip()
        if "|" in line and len(line.split("|")) >= 4:
            if current:
                commits.append(current)
            parts = line.split("|")
            current = {
                "hash": parts[0],
                "author": parts[1],
                "date": parts[2],
                "message": "|".join(parts[3:]),
                "files": [],
            }
        elif line and current and "\t" in line:
            parts = line.split("\t")
            if len(parts) == 3:
                adds = int(parts[0]) if parts[0] != "-" else 0
                dels = int(parts[1]) if parts[1] != "-" else 0
                current["files"].append({
                    "path": parts[2], "adds": adds, "deletes": dels,
                })

    if current:
        commits.append(current)

    return commits

commits = parse_git_log(REPO_PATH)
print(f"Sparsowano {len(commits)} commitów")

In [ ]:
def compute_process_metrics(commits: list[dict], py_files: list[str]) -> pd.DataFrame:
    """Oblicz metryki procesowe per plik."""
    # TODO: Zaimplementuj obliczanie metryk procesowych
    # Dla każdego pliku z py_files zbierz:
    # - churn (suma adds + deletes ze wszystkich commitów)
    # - num_commits (ile commitów dotknęło tego pliku)
    # - num_authors (ilu unikatowych autorów)
    # - age_days (różnica między dzisiejszą datą a datą najstarszego commitu z tym plikiem)
    #
    # Wskazówka: iteruj po commitach i ich plikach,
    # zbieraj dane w defaultdict, potem zamień na DataFrame
    pass

df_process = compute_process_metrics(commits, py_files)
print(f"Zebrano metryki procesowe dla {len(df_process)} plików")
df_process.head(10)

## 5. Etykietowanie: buggy vs clean

Plik jest "buggy" jeśli był zmieniany w commicie z wiadomością zawierającą słowa kluczowe sugerujące naprawę błędu.

In [ ]:
def label_buggy_files(commits: list[dict], py_files: list[str]) -> dict[str, int]:
    """Oznacz pliki jako buggy (1) lub clean (0)."""
    # TODO: Zaimplementuj etykietowanie
    # Dla każdego commitu sprawdź czy wiadomość zawiera BUG_KEYWORDS
    # Jeśli tak, oznacz wszystkie pliki .py z tego commitu jako buggy
    # Zwróć dict: filename -> 1 (buggy) lub 0 (clean)
    pass

labels = label_buggy_files(commits, py_files)
buggy_count = sum(labels.values())
clean_count = len(labels) - buggy_count
print(f"Buggy: {buggy_count}, Clean: {clean_count}")
print(f"Stosunek buggy: {buggy_count / len(labels) * 100:.1f}%")

## 6. Złączenie w jeden dataset

In [ ]:
# Złącz metryki produktowe i procesowe
df = df_product.merge(df_process, on="filename", how="inner")

# Dodaj etykiety
df["is_buggy"] = df["filename"].map(labels).fillna(0).astype(int)

print(f"Finalny dataset: {len(df)} wierszy, {len(df.columns)} kolumn")
print(f"\nKolumny: {list(df.columns)}")
print(f"\nBuggy: {df['is_buggy'].sum()}, Clean: {(df['is_buggy'] == 0).sum()}")
df.head(10)

In [ ]:
# Zapisz dataset
df.to_csv("dataset.csv", index=False)
print("Zapisano dataset.csv")

## 7. Eksploracja danych

### 7.1 Statystyki opisowe

In [ ]:
df.describe()

### 7.2 Rozkłady cech (histogramy)

In [ ]:
features = ["loc", "avg_cc", "max_cc", "churn", "num_commits", "num_authors", "age_days"]

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, feat in enumerate(features):
    axes[i].hist(df[feat], bins=20, edgecolor="black", alpha=0.7)
    axes[i].set_title(feat)
    axes[i].set_ylabel("Liczba plików")

# Ukryj ostatni pusty wykres
axes[-1].set_visible(False)

plt.suptitle("Rozkłady cech", fontsize=14)
plt.tight_layout()
plt.show()

### 7.3 Balans klas

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

# Wykres kołowy
counts = df["is_buggy"].value_counts()
ax1.pie(counts, labels=["Clean", "Buggy"], autopct="%1.1f%%",
        colors=["#2ecc71", "#e74c3c"])
ax1.set_title("Balans klas")

# Wykres słupkowy
counts.plot(kind="bar", ax=ax2, color=["#2ecc71", "#e74c3c"])
ax2.set_title("Liczba plików per klasa")
ax2.set_xticklabels(["Clean (0)", "Buggy (1)"], rotation=0)
ax2.set_ylabel("Liczba plików")

plt.tight_layout()
plt.show()

### 7.4 Macierz korelacji

In [ ]:
plt.figure(figsize=(10, 8))
corr = df[features + ["is_buggy"]].corr()
sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu_r",
            center=0, vmin=-1, vmax=1)
plt.title("Macierz korelacji")
plt.tight_layout()
plt.show()

### 7.5 Boxploty: buggy vs clean

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, feat in enumerate(features):
    sns.boxplot(data=df, x="is_buggy", y=feat, ax=axes[i],
                palette=["#2ecc71", "#e74c3c"])
    axes[i].set_title(feat)
    axes[i].set_xticklabels(["Clean", "Buggy"])

axes[-1].set_visible(False)

plt.suptitle("Porównanie cech: Buggy vs Clean", fontsize=14)
plt.tight_layout()
plt.show()

## 8. Odpowiedzi na pytania

Odpowiedzcie na poniższe pytania (zamieńcie TODO na odpowiedzi):

**1. Ile plików .py znaleźliście? Ile z nich jest buggy, a ile clean?**

TODO: Twoja odpowiedź

**2. Czy dataset jest zbalansowany? Jaki jest stosunek buggy:clean?**

TODO: Twoja odpowiedź

**3. Które cechy najbardziej różnią się między buggy a clean?**

TODO: Twoja odpowiedź (patrz boxploty powyżej)

**4. Czy widzicie korelacje między cechami? Które cechy są ze sobą skorelowane?**

TODO: Twoja odpowiedź (patrz macierz korelacji)

**5. Czy heurystyka etykietowania jest idealna? Jakie są jej wady?**

TODO: Twoja odpowiedź